**Big Data**

**2020-2021**

**Programming Assignment # 1**

**Guidelines:**

1. The assignment should be performed individually.
2. All submissions will be online (e-learning).
3. Assignment should include your full name and student ID.
4. Cutoff date for this homework is 05-1-2021 @ 12:00 PM.
5. Any late submission will be graded 0/100.
6. Academic Fraud: Cases of plagiarism will be handled according to university regulations.
7. Your homework should be solved as a Jupyter notebook file
8. You will submit a .ipynb file called YourName-ID.ipynb
9. Show all code in each step.



---------------------

## Question 1 [50/100]

Download **Mosquitoes-Trap-Data** from this [URL](https://data.edmonton.ca/Environmental-Services/Mosquitoes-Trap-Data/5zeu-wkpv).

Write a complete PySpark application to answer the follwoing question. Make sure to define the schema and prvide all necessary spark transofrmation and action code to get to the answers.   
  
  - **Q1.1** Report all problems you can find in the dataset and how you fixed them.
  - **Q1.2** How many mosquitos caught in 2014?
  - **Q1.3** How many mosquitos of each species were caught?
  - **Q1.4** Which traps caught the most mosquitos?
  



In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F

# Define Schema 
Schema= StructType([
    StructField("TRAP_DATE", StringType(), True),
    StructField("GENUS", StringType(), True),
    StructField("SPECIES", StringType(), True),
    StructField("TYPE", StringType(), True),
    StructField("GENDER", StringType(), True),
    StructField("RURAL NORTH WEST", IntegerType(), True),
    StructField("RURAL NORTH EAST", IntegerType(), True),
    StructField("RURAL SOUTH EAST", IntegerType(), True),
    StructField("RIVER VALLEY EAST", IntegerType(), True),
    StructField("RIVER VALLEY WEST", IntegerType(), True),
    StructField("RESIDENTIAL NORTH", IntegerType(), True),
    StructField("RURAL SOUTH WEST", IntegerType(), True),
    StructField("LAGOON", IntegerType(), True),
    StructField("GOLF COURSE", IntegerType(), True),
    StructField("INDUSTRIAL PARK", IntegerType(), True),
    StructField("RESIDENTIAL SOUTH", IntegerType(), True),
    StructField("TOTAL", IntegerType(), True),
])


# Data file path 
location ="data/Mosquitoes_Trap_Data__May_2011_to_Sep_2015_.csv"

# Spark session 
spark = SparkSession.builder.appName('BD_Assignemnet_1').master("local[*]").getOrCreate()

# reading data 

data = spark.read.csv(path=location, sep=",", header=True,schema=Schema)


## Q1.1 Report all problems you can find in the dataset and how you fixed them.

> **Show some rows**

In [2]:
data.sample(0.01).toPandas().head(3).T

,0,1,2
TRAP_DATE,May 24 2011,May 31 2011,July 19 2011
GENUS,None,Culex,Culiseta
SPECIES,None,territans,impatiens
TYPE,None,None,None
GENDER,Male,Female,Female
RURAL NORTH WEST,69,NaN,NaN
RURAL NORTH EAST,28,NaN,NaN
RURAL SOUTH EAST,25,NaN,NaN
RIVER VALLEY EAST,1,NaN,NaN
RIVER VALLEY WEST,1,NaN,NaN


> **Stats summary**

In [3]:
data.summary().toPandas().T

,0,1,2,3,4,5,6,7
summary,count,mean,stddev,min,25%,50%,75%,max
TRAP_DATE,4386,None,None,August 02 2011,None,None,None,September 29 2015
GENUS,4182,None,None,Aedes,None,None,None,Mansonia
SPECIES,3672,None,None,alaskaensis,None,None,None,vexans
TYPE,2958,None,None,Banded legs,None,None,None,Unknown
GENDER,4386,None,None,Female,None,None,None,Male
RURAL NORTH WEST,582,25.008591065292098,123.10009470659337,0,1,2,9,1752
RURAL NORTH EAST,517,76.41586073500967,447.4715693838706,0,1,3,20,8355
RURAL SOUTH EAST,431,18.48027842227378,103.731782736487,0,1,3,10,1839
RIVER VALLEY EAST,386,18.10103626943005,67.542330345745,0,1,3,11,1117


In [4]:
data.columns 

['TRAP_DATE',
 'GENUS',
 'SPECIES',
 'TYPE',
 'GENDER',
 'RURAL NORTH WEST',
 'RURAL NORTH EAST',
 'RURAL SOUTH EAST',
 'RIVER VALLEY EAST',
 'RIVER VALLEY WEST',
 'RESIDENTIAL NORTH',
 'RURAL SOUTH WEST',
 'LAGOON',
 'GOLF COURSE',
 'INDUSTRIAL PARK',
 'RESIDENTIAL SOUTH',
 'TOTAL']

> Duplicated rows and null counts 

In [5]:

for col in data.columns: 
    print("---------------------------------------------------")
    print(f"Total records in {col}",data.select(col).count())
    print(f"Total nulls in {col}",data.select(col).dropna().count())

print("---------------------------------------------------")

print("Total number of duplicated rows ",data.count() - data.drop_duplicates().count())


---------------------------------------------------
Total records in TRAP_DATE 4387
Total nulls in TRAP_DATE 4386
---------------------------------------------------
Total records in GENUS 4387
Total nulls in GENUS 4182
---------------------------------------------------
Total records in SPECIES 4387
Total nulls in SPECIES 3672
---------------------------------------------------
Total records in TYPE 4387
Total nulls in TYPE 2958
---------------------------------------------------
Total records in GENDER 4387
Total nulls in GENDER 4386
---------------------------------------------------
Total records in RURAL NORTH WEST 4387
Total nulls in RURAL NORTH WEST 582
---------------------------------------------------
Total records in RURAL NORTH EAST 4387
Total nulls in RURAL NORTH EAST 517
---------------------------------------------------
Total records in RURAL SOUTH EAST 4387
Total nulls in RURAL SOUTH EAST 431
---------------------------------------------------
Total records in RIVER VA

# Q1.2 How many mosquitos caught in 2014?

In [37]:
dataQ12 = data.withColumn('YEAR',year(to_date(data.TRAP_DATE, 'MMMM dd yyyy'))).select('YEAR','TOTAL')
Total_2014=dataQ12.groupBy("YEAR").sum("TOTAL").where(dataQ12.YEAR==2014).collect()[0][1]
print(f" Mosquitos caught in 2014 = {Total_2014}")

 Mosquitos caught in 2014 = 33673


## Q1.3 How many mosquitos of each species were caught?

In [49]:

for result in data.groupBy(data.SPECIES).sum('TOTAL').collect():
    print(f'Species = {result[0]} , TOTAL = {result[1]}')


Species = intrudens , TOTAL = 7
Species = increpitus , TOTAL = 8
Species = punctor , TOTAL = 0
Species = earlei , TOTAL = 523
Species = hexodontus , TOTAL = 0
Species = dorsalis , TOTAL = 1124
Species = minnesotae , TOTAL = 197
Species = impiger , TOTAL = 0
Species = mercurator , TOTAL = 2
Species = cataphylla , TOTAL = 0
Species = riparius , TOTAL = 104
Species = None , TOTAL = 28010
Species = spencerii , TOTAL = 923
Species = provocans , TOTAL = 1
Species = pionips , TOTAL = 0
Species = alaskensis , TOTAL = 141
Species = tarsalis , TOTAL = 76
Species = inornata , TOTAL = 11806
Species = alaskaensis , TOTAL = 8
Species = cinereus , TOTAL = 1
Species = campestris , TOTAL = 45
Species = flavescens , TOTAL = 953
Species = fitchii , TOTAL = 283
Species = impatiens , TOTAL = 55
Species = incidens , TOTAL = 79
Species = perturbans , TOTAL = 157
Species = communis , TOTAL = 1
Species = nigromaculis , TOTAL = 12
Species = excrucians , TOTAL = 193
Species = pullatus , TOTAL = 0
Species = terri

## Q1.4 Which traps caught the most mosquitos?

In [82]:
Traps=['RURAL NORTH WEST',
 'RURAL NORTH EAST',
 'RURAL SOUTH EAST',
 'RIVER VALLEY EAST',
 'RIVER VALLEY WEST',
 'RESIDENTIAL NORTH',
 'RURAL SOUTH WEST',
 'LAGOON',
 'GOLF COURSE',
 'INDUSTRIAL PARK',
 'RESIDENTIAL SOUTH']

for trap in Traps:
    sumCaught=data.agg(sum(trap)).collect()[0][0]
    print(f'{trap} caught {sumCaught}')
print("-------------------------------")
print ("RURAL NORTH EAST caught 39507")
print("-------------------------------")


RURAL NORTH WEST caught 14555
RURAL NORTH EAST caught 39507
RURAL SOUTH EAST caught 7965
RIVER VALLEY EAST caught 6987
RIVER VALLEY WEST caught 5501
RESIDENTIAL NORTH caught 2712
RURAL SOUTH WEST caught 25660
LAGOON caught 5678
GOLF COURSE caught 1788
INDUSTRIAL PARK caught 2789
RESIDENTIAL SOUTH caught 7796
-------------------------------
RURAL NORTH EAST caught 39507
-------------------------------


-----------------

## Question 2 [50/100]

Download **LED Streetlight Conversion Data** from this [URL](https://data.edmonton.ca/Transportation/LED-Streetlight-Conversion/rxke-mcvd).

Write a complete PySpark application to answer the follwoing questions. Make sure to define the schema and provide all necessary spark transofrmations and actions  to get to the answers.   
  
  - **Q2.1** Report all problems you can find in the dataset and how you fixed them.
  - **Q2.2** How many total streetlights?
  - **Q2.3** How many streetlights are converted to LED?
  - **Q2.4** How many streetlights were converted by year?
  

In [117]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import pyspark.sql.functions as F


# Define Schema 
Schema = StructType([
    StructField("Type", StringType(), True),
    StructField("Watts", IntegerType(), True),
    StructField("Start Date", StringType(), True),
    StructField("Ownership", StringType(), True),
    StructField("Latitude", StringType(), True),
    StructField("Longitude", StringType(), True),
    StructField("Location", StringType(), True)
])

#Define Path to file 

location ="data/LED_Streetlight_Conversion.csv"

# Spark session 
spark = SparkSession.builder.appName('BD_Assignemnet_1_2').master("local[*]").getOrCreate()

# reading data 

data = spark.read.csv(path=location, sep=",", header=True,schema=Schema)




## Q2.1 Report all problems you can find in the dataset and how you fixed them.


> **Show some rows**

In [118]:
data.sample(0.01).toPandas().head(3).T

,0,1,2
Type,LED,LED,LED
Watts,32,43,35
Start Date,September 01 2018,June 05 2012,September 01 2018
Ownership,COE,COE,COE
Latitude,53.449710774166746,53.51411334717685,53.45158504970245
Longitude,-113.66792512698021,-113.51711129147948,-113.66662813639026
Location,"(5924127.8, 22060.0)","(5931351.954, 32030.081)","(5924336.77706278, 22145.184908874)"


> **Stats summary**

In [119]:
data.summary().toPandas().T

,0,1,2,3,4,5,6,7
summary,count,mean,stddev,min,25%,50%,75%,max
Type,117374,None,None,HPS,None,None,None,MV
Watts,117341,105.95265934328155,80.14544144710132,14,60,86,150,1000
Start Date,117373,None,None,April 01 1988,None,None,None,September 30 2016
Ownership,117374,None,None,COE,None,None,None,COE
Latitude,117374,53.5264394084167,0.060826207414548666,53.33809790153047,53.477997312079076,53.530339754730285,53.57269520597053,53.709033019147036
Longitude,117374,-113.51068284568578,0.07784270909966953,-113.29590753390248,-113.56655003079273,-113.50416741950603,-113.45028577060721,-113.71358103352117
Location,117374,None,None,"(5911740.64670252, 28396.6410121975)",None,None,None,"(5953089.32, 38012.36)"


In [120]:
data.columns 

['Type',
 'Watts',
 'Start Date',
 'Ownership',
 'Latitude',
 'Longitude',
 'Location']

> **Duplicated rows and null counts** 

In [121]:

for col in data.columns: 
    print("---------------------------------------------------")
    print(f"Total records in {col}",data.select(col).count())
    print(f"Total nulls in {col}",data.select(col).dropna().count())

print("---------------------------------------------------")

print("Total number of duplicated rows ",data.count() - data.drop_duplicates().count())


---------------------------------------------------
Total records in Type 117374
Total nulls in Type 117374
---------------------------------------------------
Total records in Watts 117374
Total nulls in Watts 117341
---------------------------------------------------
Total records in Start Date 117374
Total nulls in Start Date 117373
---------------------------------------------------
Total records in Ownership 117374
Total nulls in Ownership 117374
---------------------------------------------------
Total records in Latitude 117374
Total nulls in Latitude 117374
---------------------------------------------------
Total records in Longitude 117374
Total nulls in Longitude 117374
---------------------------------------------------
Total records in Location 117374
Total nulls in Location 117374
---------------------------------------------------
Total number of duplicated rows  2447


**Drop duplicated**

In [122]:
data=data.drop_duplicates()

print("Total number of duplicated rows ",data.count() - data.drop_duplicates().count())


Total number of duplicated rows  0


## Q2.2 How many total streetlights?

In [123]:
print(f"Total streetlights = {data.count()}")

Total streetlights = 114927


## Q2.3 How many streetlights are converted to LED?

In [124]:
data.select('Type').distinct().show(3)

+----+
|Type|
+----+
| INC|
| LED|
| HPS|
+----+
only showing top 3 rows



In [129]:
data.select('Type').filter(data['Type']=='LED').show(3)

+----+
|Type|
+----+
| LED|
| LED|
| LED|
+----+
only showing top 3 rows



In [155]:

# Get all instances with LED type 
LED_data=data.select(['Type','Start Date' ,'Location' ]).where(data["Type"]=="LED")

# Count the instances based on location 
LED_count=data.groupBy("Location").count()

# Keep the instances with the same location woth more than one type reported 
LED_count=LED_count.filter(LED_count['count']>1)

# Use innerjoin to identify instances with more than one type and one of them is LED 

LED_count= LED_count.join(LED_data,LED_count.Location ==  LED_data.Location,"inner")

print(f" No. of street lights converted to LED = {LED_count.count()}")

 No. of street lights converted to LED = 328


## Q2.4 How many streetlights were converted by year?

In [223]:
# Create year column

split_col = split(data['Start Date'], ' ')
data = data.withColumn('month', split_col.getItem(0))
data = data.withColumn('day', split_col.getItem(1))
data = data.withColumn('year', split_col.getItem(2))

# Get all good years 
List_of_years=list(data.select('year').dropna().filter(data['year'] < '2020').distinct().toPandas()['year'])
List_of_years=sorted([int(i) for i in List_of_years])
List_of_years=[str(i) for i in List_of_years]
for year_i in List_of_years:
    # Count the instances based on location at certain year (year_i)
    LED_count_year=data.filter(data['year']==year_i).groupBy("Location").count()

    # Keep the instances with the same location with more than one type reported. Represent the places that has been converted  
    LED_count_year_converted=LED_count_year.filter(LED_count_year['count']>1).count()
    print("====================================")
    print(f'In Year {year_i} they converted {LED_count_year_converted} LED' )
    print("====================================")




In Year 1974 they converted 0 LED
In Year 1976 they converted 0 LED
In Year 1977 they converted 0 LED
In Year 1978 they converted 0 LED
In Year 1979 they converted 0 LED
In Year 1980 they converted 0 LED
In Year 1981 they converted 0 LED
In Year 1982 they converted 1 LED
In Year 1983 they converted 0 LED
In Year 1984 they converted 0 LED
In Year 1985 they converted 0 LED
In Year 1986 they converted 0 LED
In Year 1987 they converted 0 LED
In Year 1988 they converted 0 LED
In Year 1989 they converted 0 LED
In Year 1990 they converted 0 LED
In Year 1991 they converted 0 LED
In Year 1992 they converted 3 LED
In Year 1993 they converted 0 LED
In Year 1994 they converted 0 LED
In Year 1995 they converted 10 LED
In Year 1996 they converted 6 LED
In Year 1997 they converted 12 LED
In Year 1998 they converted 0 LED
In Year 1999 they converted 2 LED
In Year 2000 they converted 0 LED
In Year 2001 they converted 0 LED
In Year 2002 they converted 14 LED
In Year 2003 they converted 2 LED
In Year 200